In [1]:
!pip install pandas requests

In [2]:
import requests
import json
import pandas as pd

# The direct URL to the JSON file on your MinIO console
DATA_URL = 'http://localhost:9001/browser/spotify-sample/mpd.slice.0-999.json'

try:
    # 1. Fetch the data from the URL
    response = requests.get(DATA_URL)
    response.raise_for_status() # Check for HTTP errors (4xx or 5xx)

    # 2. Parse the JSON content
    data = response.json()
    
    print("Data successfully fetched and parsed!")
    
except requests.exceptions.RequestException as e:
    print(f"Error accessing the MinIO server or file: {e}")
    print("Ensure MinIO is running and accessible at http://localhost:9001.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print("The file might not be valid JSON.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error accessing the MinIO server or file: HTTPConnectionPool(host='localhost', port=9001): Max retries exceeded with url: /browser/spotify-sample/mpd.slice.0-999.json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc6e163b790>: Failed to establish a new connection: [Errno 111] Connection refused'))
Ensure MinIO is running and accessible at http://localhost:9001.


In [2]:
!pip install minio


In [9]:
from minio import Minio
import json

# Khởi tạo client
client = Minio("host.docker.internal:9000", access_key="minioadmin", secret_key="minioadmin", secure=False)

# Tải file JSON từ bucket
data = client.get_object("spotify-sample", "mpd.slice.0-999.json")

# Đọc nội dung JSON
playlist_data = json.load(data)
print(len(playlist_data["playlists"]))


1000


In [11]:
playlists = playlist_data['playlists']
records = []

for p in playlists:
    for t in p['tracks']:
        records.append({
            'playlist_id': p['pid'],
            'playlist_name': p.get('name', ''),
            'track_name': t.get('track_name', ''),
            'artist_name': t.get('artist_name', ''),
            'album_name': t.get('album_name', '')
        })

df = pd.DataFrame(records)

print("✅ Số lượng playlist:", df['playlist_id'].nunique())
print("✅ Tổng số track entries:", len(df))
print("✅ Số lượng track duy nhất:", df['track_name'].nunique())
print(df.head())

✅ Số lượng playlist: 1000
✅ Tổng số track entries: 67503
✅ Số lượng track duy nhất: 30049
   playlist_id playlist_name                                  track_name  \
0            0    Throwbacks  Lose Control (feat. Ciara & Fat Man Scoop)   
1            0    Throwbacks                                       Toxic   
2            0    Throwbacks                               Crazy In Love   
3            0    Throwbacks                              Rock Your Body   
4            0    Throwbacks                                It Wasn't Me   

         artist_name                                    album_name  
0      Missy Elliott                                  The Cookbook  
1     Britney Spears                                   In The Zone  
2            Beyoncé  Dangerously In Love (Alben für die Ewigkeit)  
3  Justin Timberlake                                     Justified  
4             Shaggy                                      Hot Shot  


In [12]:
avg_tracks = df.groupby('playlist_id')['track_name'].count().mean()
print(f"Số bài hát trung bình mỗi playlist: {avg_tracks:.2f}")

# Nghệ sĩ phổ biến nhất
top_artists = df['artist_name'].value_counts().head(10)
print("\nTop 10 nghệ sĩ phổ biến nhất:")
print(top_artists)

# Album phổ biến nhất
top_albums = df['album_name'].value_counts().head(10)
print("\nTop 10 album phổ biến nhất:")
print(top_albums)

Số bài hát trung bình mỗi playlist: 67.50

Top 10 nghệ sĩ phổ biến nhất:
artist_name
Drake             939
Kanye West        415
Kendrick Lamar    385
Rihanna           350
Eminem            332
The Weeknd        296
Lil Uzi Vert      292
Ed Sheeran        285
Future            265
Chris Brown       259
Name: count, dtype: int64

Top 10 album phổ biến nhất:
album_name
Views                        234
Stoney                       184
DAMN.                        152
Greatest Hits                149
Beauty Behind The Madness    146
Original Album Classics      138
Blurryface                   134
More Life                    127
American Teen                122
The Life Of Pablo            121
Name: count, dtype: int64
